# Investigating potential regulatory relationships between TFs in breast cancer using DRAGON

Author: Deborah Weighill<sup>1</sup>

<sup>1</sup> Department of Biostatistics, Harvard T.H. Chan School of Public Health, Boston, MA.

# 1. Introduction
In this tutorial, we investigate potential regulatory relationships between transcription factors (TFs) based on their methylation profiles as well as their gene expression. For this analysis we use the [DRAGON](https://netzoo.github.io/zooanimals/dragon/)<sup>1</sup> algorithm **(Determining Regulatory Associations using Graphical models on multi-Omic Networks)** to calculate partial correlations between expression and methylation profiles of TFs. DRAGON calibrates its parameters to achieve an optimal trade-off between the network's complexity and estimation accuracy, while  explicitly accounting for the characteristics of each of the assessed omics layers. DRAGON is distributed through the Network Zoo package (netZooPy v0.8; [netzoo.github.io](netzoo.github.io)) in Python. Recently, we added an R implementation of DRAGON that is available in netZooR v1.2.1.
However, for this tutorial and because we need to use TCGA2BED software to process the data, we coded a function that imports DRAGON from Python to R. 

# 2. Comparing DRAGON networks and GGMs

In theis section, we will compare DRAGON to standard Gaussian Graphical Models (GGMs) and show that DRAGON networks can capture biologically meaningful associations.

## Load libraries
First, we start by loading the libraries.

In [ ]:
library('dplyr')    # For data processing
library('tidyr')    # For data processing
library('RcppCNPy')
library('ggplot2')  # For plotting
library('ggthemes') # To set ggplot themes
library('visNetwork')   # To visualize the network
library('RColorBrewer') # To set the color palette
source("./dragonScripts/call_dragon.R") # Script to load Dragon from netZooPy and bind to Python

Then, we need to set the paths for the files on the netbooks server.

In [ ]:
ppath = '/opt/data/netZooPy/dragon/'

## Create input matrices
We construct our input matrices for DRAGON. We obtained TCGA breast cancer expression and methylation data which had already been preprocessed as `.bed` files from TCGA2BED<sup>2</sup> (see http://bioinf.iasi.cnr.it/tcga2bed/) and used the TCGA2BED software to create a combined methylation and gene expression matrix. From this combined matrix, we perform the following preprocessing steps:

1. Create a gene expression matrix removing genes with consistently low expression
2. Scale the expression matrix
3. Create a methylation matrix which removes unknown methylation values.
4. Scale the methylation matrix.
5. Down-select matrices to TFs.

First, we start by loading the combined gene expression and methylation data matrix.

In [ ]:
data <- read.table(paste0(ppath,"combined_RNASeq_Meth_matrix_BRCA.txt"), header = TRUE, row.names = 1)
meth <- t(data[,c(1:20049)])
expr <- t(data[,c(20050:40534)])

For gene expression, we emove low-expressed genes, which are rsem normalized, and then scale them.

In [ ]:
exprLow <- apply(expr , MARGIN = 1, FUN = function(x) length(x[(is.na(x)| x<=1) ]))
cleanExpr <- expr[exprLow <= (0.2*ncol(expr)),]
scaled_expr <- scale(t(cleanExpr))

For this data set, we have 15,733 genes across 872 samples. For methylation, we remove genes that are annotated as "?", and then scale the data frame.

In [ ]:
methNA <- apply(meth , MARGIN = 1, FUN = function(x) length(which(x == "?")))
cleanMeth <- meth[methNA == 0,]
write.table(cleanMeth, file = "../data/cleanMeth.txt", sep = "\t", quote = FALSE, row.names = TRUE, col.names = TRUE)
cleanMeth2 <- read.table("../data/cleanMeth.txt", header = TRUE, row.names = 1)
scaled_meth <- scale(t(cleanMeth2))

For methylation, we have 19830 genes across 872 samples. Finally, we extract the TF names from methylation data.

In [ ]:
tfs_meth <- read.table(paste0(ppath,"meth_tf_ids.txt"), header = FALSE)
methylation_subset <- scaled_meth[,which(colnames(scaled_meth) %in% tfs_meth$V1)]

And TF names from expression data.

In [ ]:
tfs_expr <- read.table(paste0(ppath,"rnaseq_tf_ids.txt"), header = FALSE)
expression_subset <- scaled_expr[,which(colnames(scaled_expr) %in% tfs_expr$V1)]

## Run DRAGON
Now we run DRAGON, which will calculate partial correlations, p-values and adjusted p-values for pairs of TFs based on their expression and methylation profiles.

In [ ]:
XA0 <- expression_subset
XB0 <- methylation_subset

DRAGON reports the partial correlations, p-values and adjusted p-values betweene each pair of TFs (i,j).

In [ ]:
res2 <- call.dragon(XA0, XB0)

The `AA` result contains partial correlations between the expression profile of TF i and the expression profile of TF j: 

In [ ]:
head(res2$resAA)

the `BB` result contains partial correlations between the methylation profile of TF i and the methylation profile of TF j:

In [ ]:
head(res2$resAB)

the `AB` result contains partial correlations between the expression profile of TF i and the methylation profile of TF j:

In [ ]:
head(res2$resBB)

Then, we parse feature columns to get clean TF ids. There can be edges in the network that link 2 TFs based on their combined methylation and expression profiles:

In [ ]:
exp_meth_2layer <- res2$resAB
exp_meth_2layer <- separate(exp_meth_2layer, feature1, c("tf_rnaseq",NA), "_", remove = TRUE)
exp_meth_2layer <- separate(exp_meth_2layer, feature2, c("tf_dnameth",NA), "_", remove = TRUE)
head(exp_meth_2layer)

Edges that connect 2 TFs based on their methylation profiles:

In [ ]:
meth_meth_2layer <- res2$resBB
meth_meth_2layer <- separate(meth_meth_2layer, feature1, c("tf_dnameth1",NA), "_", remove = TRUE)
meth_meth_2layer <- separate(meth_meth_2layer, feature2, c("tf_dnameth2",NA), "_", remove = TRUE)
meth_meth_2layer$id <- paste0(meth_meth_2layer$tf_dnameth1,meth_meth_2layer$tf_dnameth2)
head(meth_meth_2layer)

Edges that connect 2 TFs based on their expression profiles:

In [ ]:
exp_exp_2layer <- res2$resAA
exp_exp_2layer <- separate(exp_exp_2layer, feature1, c("tf_rnaseq1",NA), "_", remove = TRUE)
exp_exp_2layer <- separate(exp_exp_2layer, feature2, c("tf_rnaseq2",NA), "_", remove = TRUE)
exp_exp_2layer$id <- paste0(exp_exp_2layer$tf_rnaseq1,exp_exp_2layer$tf_rnaseq2)
head(exp_exp_2layer)

## Compare DRAGON to standard GGM
Here we run standard Gaussian Graphical Model to calculate partial correlations between a single layer of expression data (i.e. no methylation information included). This will allow us to investigate if there exist TF-TF relationships that are better explained by including a methylation layer. We start first by selecting the expression subset.

In [ ]:
XA0 <- expression_subset

Then inferring the GGM:

In [ ]:
res1 <- call.GGM(XA0)
res1$res

The result has edges betwee variables in the network and the significance of association. We can save the result to process it later.

In [ ]:
saveRDS(res1,"../data/dragon_TFs_results_1layer_03102021.Rds")

Finally, we format the network in the same DRAGON annotation

In [ ]:
exp_exp_1layer <- res1$res
exp_exp_1layer <- separate(exp_exp_1layer, feature1, c("tf_rnaseq1",NA), "_", remove = TRUE)
exp_exp_1layer <- separate(exp_exp_1layer, feature2, c("tf_rnaseq2",NA), "_", remove = TRUE)
exp_exp_1layer$id <- paste0(exp_exp_1layer$tf_rnaseq1,exp_exp_1layer$tf_rnaseq2)

Now we select TF-TF relationships which are significant at an adjusted p-value of 0.05, investigate if there are significant relationships in the GGM which are altered when looking at the 2-layer DRAGON relationships. We start first by apply adjusted p-value threshold:

In [ ]:
exp_exp_1layer_0.05 <- exp_exp_1layer[which(exp_exp_1layer$adj_p_vals <= 0.05),]
exp_exp_2layer_0.05 <- exp_exp_2layer[which(exp_exp_2layer$adj_p_vals <= 0.05),]
meth_meth_2layer_0.05 <- meth_meth_2layer[which(meth_meth_2layer$adj_p_vals <= 0.05),]

Then, we identify relationships (network edges) significant in the expression GGM, non-significant in the DRAGON expression network and significant in the DRAGON methylation network.

In [ ]:
exp_exp_1layer_0.05[which(!(exp_exp_1layer_0.05$id %in% exp_exp_2layer_0.05$id) & (exp_exp_1layer_0.05$id %in% meth_meth_2layer_0.05$id)),]

Finally, we can show the p-values for associations

In [ ]:
exp_exp_1layer[which((exp_exp_1layer$tf_rnaseq1 == "ELF4") &(exp_exp_1layer$tf_rnaseq2 == "ZBTB33")),]
exp_exp_2layer[which((exp_exp_2layer$tf_rnaseq1 == "ELF4") &(exp_exp_2layer$tf_rnaseq2 == "ZBTB33")),]
meth_meth_2layer_0.05[which((meth_meth_2layer_0.05$tf_dnameth1 == "ELF4") &(meth_meth_2layer_0.05$tf_dnameth2 == "ZBTB33")),]

These results show the relationship between TFs ELF4 and ZBTB33, which appear to be correlated in terms of expression in the GGM, are in fact not significantly correlated in the DRAGON expression network, but are significantly correlated in the DRAGON methylation network. This suggets that this relationship is driven by co-methylation and not co-expression.

# 2. Finding similarities between TFs in DRAGON co-methylation network
## Investigate proximity of co-methylated genes
Now, we investigate the physical proximity of TF gene regions in the genome and how this relates to co-methylation. We start first by reading the annotation file

In [ ]:
genes <- read.table(paste0(ppath,"geneID_name_map.txt"), header = FALSE)
colnames(genes) <- c("ensID","name","chrom","start","stop")

We then calculate the midpoint base in the middle of the gemne

In [ ]:
genes$mid <- (genes$stop - genes$start)/2

Next, we identify the chromosome on which the TF's gene resides

In [ ]:
meth_meth_2layer$chrom1 <- genes$chrom[match(meth_meth_2layer$tf_dnameth1, genes$name)]
meth_meth_2layer$chrom2 <- genes$chrom[match(meth_meth_2layer$tf_dnameth2, genes$name)]

if the TF's respective genes reside on the same chromosome, we calculate the distance between the midpoints of the genes.

In [ ]:
meth_meth_2layer$distance <- ifelse(meth_meth_2layer$chrom1 == meth_meth_2layer$chrom2, abs((genes$mid[match(meth_meth_2layer$tf_dnameth1, genes$name)]) - (genes$mid[match(meth_meth_2layer$tf_dnameth2, genes$name)])), NA)

Then, we mark each pair of TFs as significantly co-methylated or not, based on DRAGON adjusted p-values.

In [ ]:
meth_meth_2layer$signif <- ifelse(meth_meth_2layer$adj_p_vals <= 0.05, "P <= 0.05","P > 0.05")

Next, we annoatate the genes and their chromosomes using the distances we computed previously.

In [ ]:
meth_meth_2layer_0.05$chrom1 <- genes$chrom[match(meth_meth_2layer_0.05$tf_dnameth1, genes$name)]
meth_meth_2layer_0.05$chrom2 <- genes$chrom[match(meth_meth_2layer_0.05$tf_dnameth2, genes$name)]
meth_meth_2layer_0.05$distance <- ifelse(meth_meth_2layer_0.05$chrom1 == meth_meth_2layer_0.05$chrom2, abs((genes$mid[match(meth_meth_2layer_0.05$tf_dnameth1, genes$name)]) - (genes$mid[match(meth_meth_2layer_0.05$tf_dnameth2, genes$name)])), NA)

Finally, we plot distance vs significance as a violin plot (for pairs of TFs on the same chromosome).

In [ ]:
p <- ggplot(meth_meth_2layer, aes(x=signif, y=distance, fill=signif)) + geom_violin(trim = FALSE)  + theme_bw() + scale_fill_manual(values=c("red", "blue","orange","cyan", "darkgreen", "violet")) + labs(x="P-value", y = "Distance between genes (bp)", title = "Gene proximity distributions") + theme(legend.position = "none")
p

To better understand the distribution of significant edges across chromosomes, we can first compute the number of significant associations

In [ ]:
num_signif <- length(meth_meth_2layer$distance[which(meth_meth_2layer$signif == "P <= 0.05")])
num_signif

Then, we count the number of non-significant associations

In [ ]:
num_non_signif <- length(meth_meth_2layer$distance[which(meth_meth_2layer$signif == "P > 0.05")])
num_non_signif

We can count the number of significant associations on different chromosomes

In [ ]:
num_signif_interChrom <- length(which(is.na(meth_meth_2layer$distance[which(meth_meth_2layer$signif == "P <= 0.05")])))
num_signif_interChrom

Finally, we can count the number of non-significant associations on different chromosomes

In [ ]:
num_Nonsignif_interChrom <- length(which(is.na(meth_meth_2layer$distance[which(meth_meth_2layer$signif == "P > 0.05")])))
num_Nonsignif_interChrom

Note that almost 95 percent of non-significant edges are across chromosomes.

## Visualize network
Now we plot the significant DRAGON relationships with expression-expression relationships colored orange, methylation-methylation edges colored green, and expression-methylation relationships colored purple. First, we set the parameters of the `visNetwork` library.

In [ ]:
mm_edges <- meth_meth_2layer_0.05[,c(4,5)]
colnames(mm_edges) <- c("from","to")
mm_edges$color <- "green"

ee_edges <- exp_exp_2layer_0.05[,c(4,5)]
colnames(ee_edges) <- c("from","to")
ee_edges$color <- "orange"

meth_exp_2layer_0.05 <- exp_meth_2layer[which(exp_meth_2layer$adj_p_vals <= 0.05),]
me_edges <- meth_exp_2layer_0.05[,c(4,5)]
colnames(me_edges) <- c("from","to")
me_edges$color <- "purple"

edges <- rbind(mm_edges, ee_edges, me_edges)
nodes <- data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))))
nodes$label <- nodes$id             

Then, we plot the final network

In [ ]:
net <- visNetwork(nodes, edges, width = "100%")
net 

This network summarizes our findings in the finding associations between TFs in breast cancer based on methylation and expression profiles.

# References
1- Weighill, Deborah, et al. "DRAGON: Determining Regulatory Associations using Graphical models on multi-Omic Networks." arXiv preprint arXiv:2104.01690 (2021).

2- Cumbo, Fabio, et al. "TCGA2BED: extracting, extending, integrating, and querying The Cancer Genome Atlas." BMC bioinformatics 18.1 (2017): 1-9.